# Demonstração básica da configuração e do Uso do SASPy

Um resumo e demonstração resumida para configuração e uso do SASPy.

Obs.: este exemplo foi montado utilizandio como servidor o SAS Grid e todos os dados e os retonos das celulas estão anonimizado não fazendo referencia a dodos reais. 

## 1. Libs Python necessárias:

In [1]:
#SASPy que viabilisa o uso do SAS em conjunto com o Python 
import saspy 

#Necessária para as saidas dos resultados do SAS
from IPython.display import HTML

#Lib de analise de dados classica 
import pandas as pd


## 2. Forma de conexão do SAS usando o SASPy

NOTA:para possibilitar a conexão com o SAS é necessário configurar o arquivo "sascfg_personal.py".
Este arquivo vem juntamente com a instalação do SASPy.
Para falicitar e recomendado copia-lo para a raz o seu projeto e o modificar por lá.
No caso específico da Oi alterar as segintes linhas:
    
    Para indicar que o SAS server esta autenticando em infra Linux (caso utilizado para este exemplo):
        SAS_config_names=['iomlinux']
    
    Para se configurar o Workspace Server específico que se deseja conectar:
        iomlinux = {'java'      : '/usr/bin/java',
            'iomhost'   : ['<servidor1_do_clusster>','<servidor2_do_clusster>','<servidor3_do_clusster>'], 
            'iomport'   : <Porta_Workspace_Server>,
            'appserver':'<Nome_do_SAS_App_Server>'
            }
   
    Para se saber qual Workspace Server estão configurados executar o código SAS:
        proc iomoperate 
            uri="iom://<servidor_de_autenticacao>:<Porta_de_autenticao>;Bridge;USER=<usuário>,PASS=<senha>";
            list DEFINED FILTER="- Workspace";
        quit; 
   
    E possível se passar como parametro qual Workspace Server se deseja conectar quando se chama o comando saspy.SASsession()
    
Mais detalhes no link: https://sassoftware.github.io/saspy/configuration.html

In [2]:
#Conecta e testa a conexão com SAS Grid
sas = saspy.SASsession()
sas

Using SAS Config named: iomlinux


Please enter the OMR user id: <usuario_sas>
Please enter the password for OMR user :  ········


SAS Connection established. Subprocess id is 7706



Access Method         = IOM
SAS Config name       = iomlinux
SAS Config file       = /sascfg_personal.py
WORK Path             = /saswork/bi/SAS_work3FD10000608C/SAS_work807D0000608C/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 5.3.0
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = latin1
Python Encoding value = latin_1
SAS process Pid value = 24716


## 3. Recuperar o nome das Libs Assinadas do SAS mapeadas no SAS Workspace Server selecionado

In [3]:
#Lista o LIBREF que deve ser usado em código SAS
SAS_GRID_APPSERVER_LIBS = sas.assigned_librefs()
SAS_GRID_APPSERVER_LIBS = sorted(SAS_GRID_APPSERVER_LIBS)
SAS_GRID_APPSERVER_LIBS

['LIB01',
 'LIB02',
 'LIB03',
 'WORK']

## 4. Usando comandos SAS

### 4.1. Usando uma comando SAS nativo e retornando o Log de execução

In [4]:
#Outra forma de listar as Lista o LIBREF que deve ser usado em código SAS
resultado = sas.submit('libname _all_ list;')
#Retorna o Log de execução
print(resultado['LOG'])


7                                                          The SAS System                             10:43 Sunday, November 5, 2023

44         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
44       ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
45         
46         libname _all_ list;
NOTE: Libref=   WORK 
      Scope=    IOM ROOT COMP ENV
      Engine=   V9
      Access=   TEMP
      Physical Name= /saswork/bi/SAS_work3FD10000608C/SAS_work807D0000608C
      Filename= /saswork/bi/SAS_work3FD10000608C/SAS_work807D0000608C
      Inode Number= 27
      Access Permission= rwx------
      Owner Name= <Usuario_Criador>
      File Size=              1KB
      File Size (bytes)= 1024
NOTE: Libref=   LIB01 
      Scope=    Object Server
      Engine=   BASE
      Physical Name= /sasdata/LIBS/LIB01
      Filename= /sasdata/LIBS/LIB01
      Inode Number= 2
      Access Permiss

### 4.2. Usando uma comando SAS nativo e retornando o LOG e resultado HTML nativo do SAS

In [5]:
#Código SAS que se deseja executar, neste caso lista o conteudo da LIB LIB03:
codigo ='''proc contents data=LIB03._all_ nods;
           run; 
           quit; '''

#Execução do código SAS
resultado = sas.submit(codigo)  

In [6]:
#Log de execução do código SAS
print(resultado['LOG']) 


9                                                          The SAS System                             10:43 Sunday, November 5, 2023

54         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
54       ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
55         
56         proc contents data=LIB03._all_ nods;
57                    run;

NOTE: PROCEDURE CONTENTS used (Total process time):
      real time           0.58 seconds
      user cpu time       0.04 seconds
      system cpu time     0.01 seconds
      memory              2201.09k
      OS Memory           26020.00k
      Timestamp           05/11/2023 10:44:25 AM
      Step Count                        3  Switch Count  0
      Page Faults                       195
      Page Reclaims                     634
      Page Swaps                        0
      Voluntary Context Switches        724
      Involuntary Context

In [7]:
#Resultado da exeucução acima 
HTML(resultado['LST'])     

### 4.3. Executa um PROC SQL e coloca o resultado na WORK

In [8]:
#Código SAS chamando um PROC SQL que criará uma tabela com o Resultado na LIB temporária WORK
codigo ='''proc sql;
               CREATE TABLE work.Teste_Work AS
               SELECT col1,
                      col2,
                      col3
                FROM LIB03.Teste_Table
               WHERE col2 between '1Sep2023:0:0:0'dt 
                              and '30Sep2023:0:0:0'dt;
            run; 
            quit; '''

#Execução do código SAS
resultado = sas.submit(codigo)

In [9]:
#Log de execução do código SAS
print(resultado['LOG']) 


11                                                         The SAS System                             10:43 Sunday, November 5, 2023

66         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
66       ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
67         
68         proc sql;
69                        CREATE TABLE work.Teste_Work AS
70                        SELECT col1,
71                               col2,
72                               col3
73                         FROM LIB03.Teste_Table
74                        WHERE col2 between '1Sep2023:0:0:0'dt
75                                       and '30Sep2023:0:0:0'dt;
NOTE: Table WORK.TESTE_WORK created, with 98080 rows and 3 columns.

76                     run;
NOTE: PROC SQL statements are executed immediately; The RUN statement has no effect.
77                     quit;
NOTE: PROCEDURE SQL used (Total proce

## 5. Recuperando os dados

###  5.1. Recupera os dados de uma tabela no SAS

In [10]:
#Recupera uma tabala de uma LIB do SAS 
dfSASTeste = sas.sasdata("Teste_Work","WORK")
dfSASTeste.head(10)

,col1,col2,col3
0,Cancelamento,2023-09-06,202310
1,Cancelamento,2023-09-19,202310
2,Cancelamento,2023-09-28,202310
3,Cancelamento,2023-09-30,202310
4,Cancelamento,2023-09-04,202310
5,Cancelamento,2023-09-01,202310
6,Cancelamento,2023-09-15,202310
7,Cancelamento,2023-09-04,202310
8,Cancelamento,2023-09-23,202310
9,Cancelamento,2023-09-29,202310


### 5.2. Se transforma o DATAFRME retornado pelo SAS para um DATAFRAME Pandas

Obs.: Esta ação de se passar para um dataframe pandas o set de dados que se deseja trbalhar é recomendada para se obter uma possível performance melhor em analise e se ter as facilidades do Python, isto considerando que o dado foi trabalhando no SAS e se esta trazendo para o dataframe Pandas somente os dados necessários para se analisar.

In [11]:
#Transforma um dataframe do SASPy para um dataframe Pandas
dfPDTeste = dfSASTeste.to_df()
dfSASTeste.head(10)

,col1,col2,col3
0,Cancelamento,2023-09-06,202310
1,Cancelamento,2023-09-19,202310
2,Cancelamento,2023-09-28,202310
3,Cancelamento,2023-09-30,202310
4,Cancelamento,2023-09-04,202310
5,Cancelamento,2023-09-01,202310
6,Cancelamento,2023-09-15,202310
7,Cancelamento,2023-09-04,202310
8,Cancelamento,2023-09-23,202310
9,Cancelamento,2023-09-29,202310


### 5.3. Exportação de reultado proveniente do SAS

Obs.: Com o reultado de uma tabela SAS em um dataframe Pandas e possível executar todas as ações que o Pandas oferece inclusive exportar o resultado.

In [12]:
dfPDTeste.to_csv('out_teste_saspy.zip')

## 6. Importando um dado externo (CSV) no SAS (WORK) 

In [13]:
#Recupera os dados de um CSV para um dataframe Pandas
dfPDTeste2 = pd.read_csv('out_teste_saspy.zip',low_memory=False)

#Leva o Dataframe Pandas para a Work do SAS (Obs.: table será o nome da tabela na Work do SAS)
dfSASTeste2 = sas.df2sd(dfPDTeste,table = 'Teste_Python_To_SAS') 

#Descreve a tabela que foi importada para a Work do SAS
codigo ='''proc contents data=work._all_ nods;
           run; 
           quit; '''

#Execução do código SAS
resultado = sas.submit(codigo) 

#Resultado da exeucução acima 
HTML(resultado['LST']) 

## 7. Magic command para execução direta de comandos SAS em uma celula do Jupyter Notebook 

Obs.: para usar o Magic command %%SAS ele solicitará conectar novamente, sendo assim a WORK será para esta sessão e as tabelas criadas em outras sessões não serão visiveis.

In [14]:
%%SAS
PROC SQL;
 SELECT COUNT(*)
 FROM LIB03.TBAEL_CONTAGEM;
RUN;

Using SAS Config named: iomlinux


Please enter the OMR user id:  <usuario_sas>
Please enter the password for OMR user :  ········


SAS Connection established. Subprocess id is 7811



_c0
209387


## Liks das referencias

https://sassoftware.github.io/saspy/index.html

https://support.sas.com/resources/papers/proceedings19/3238-2019.pdf

https://support.sas.com/resources/papers/proceedings19/3189-2019.pdf

https://blogs.sas.com/content/sasdummy/2017/04/08/python-to-sas-saspy/

https://support.sas.com/ondemand/saspy.html